In [1]:
from nlp_rake import rake
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
#from gensim.models import word2vec
import numpy as np
import pandas as pd

In [2]:
df_std = pd.read_csv('anom_detection/df_anom_std.csv')

In [34]:
stoppath = 'new_stoplist.txt'
#stoppath = 'SmartStoplist.txt'
#stoppath = 'FoxStoplist.txt'
rake_object = rake.Rake(stoppath, 5, 3, 4)


def onlySentences(article):
    new_article = ""
    for sentence in sent_tokenize(article):
        words_list = word_tokenize(sentence.lower())
        if dist in words_list:
            new_article += sentence 
            new_article += '\n'
    return new_article

def pureText(temp_article):
    article = temp_article[0].split('\n')[2]
    article += '\n'
    for article_full in temp_article[1:]:
        if len(article_full.split('\n'))> 2:
            article += article_full.split('\n')[3]
            article += '\n'
    return article

def articleDate(article):
    try:
        and_idx1 = article.find("|&| ")
        and_idx2 = article[and_idx1+4:].find("|&| ") + and_idx1 + 4
        and_idx3 = article.rfind("|&| ")

        article_date = article[and_idx2+4:and_idx3]

        dash_idx1 = article_date.find('-')

        dash_idx2 = article_date.rfind('-')

        year = int(article_date[:dash_idx1])
        month = int(article_date[dash_idx1+1:dash_idx2])
        day = int(article_date[dash_idx2+1:])

        return [article_date,year,month,day]

    except ValueError:
        pass

def do_lmtz(temp_article):
    lmtz = WordNetLemmatizer()
    words = word_tokenize(temp_article)
    new_article = ""
    for word in words:
        new_word = lmtz.lemmatize(word,'v')
        new_article = new_article + " " + new_word
    return new_article

def finalSet(article_list,start_idx,end_idx):
    
    text = pureText(article_list[start_idx:end_idx])
    sents = onlySentences(text)
    #lmtzd_sent = do_lmtz(sents)
    #lmtzd_text = do_lmtz(text)

    article = text
    return article

def extractKeywords(article_list,start_idx,min_articles,number_keywords_want):
    articles_taken = min_articles

    end_idx = start_idx+articles_taken
    end_idx = min(end_idx,len(article_list)-1)

    article = finalSet(article_list,start_idx,end_idx)

    while(len(rake_object.run(article)) < number_keywords_want) and end_idx < len(article_list)-1:
        articles_taken += 1
        end_idx = start_idx+articles_taken

        article = finalSet(article_list,start_idx,end_idx)

    start_idx += articles_taken
    keywords = rake_object.run(article)
    print len(keywords)
    print
    keyword_0 = []
    for keyword in keywords:
        keyword_0.append(keyword[0])
        print keyword

    return [keywords,keyword_0]

def extractKeywords2(article_list,start_idx,end_idx):
    article = finalSet(article_list,start_idx,end_idx)
    print 'now sending the article to rake to extract keywords.... '
    keywords = rake_object.run(article)
    print 'now making the keywords arranged for you'
    #print keywords
    keyword_0 = []
    for keyword in keywords:
        keyword_0.append(keyword[0])
    return [keywords,keyword_0,article]


def articleList():
    date_ = str(year) + '-'
    if month<10:
        date_ += '0'+str(month)
    else:
        date_ += str(month)

    folder_name = dist+'/txt_files/'
    f_name = folder_name+dist + '_' + date_ + '.txt'
    f = open(f_name,'r')
    articles = f.read()

    article_list = articles.split('\n\n')
    return article_list

def countDays(article_list):
    curr_idx = 0
    count_dayAr = []
    for day in range(1,32):
        itr_day = day
        count_day = 0
        start_idx = curr_idx
        while(itr_day<day+1):
            curr_idx += 1
            if curr_idx>=len(article_list):
                break;
            article = article_list[curr_idx]
            if len(article) > 0:
                count_day += 1
                itr_day =  articleDate(article)[3]
        count_dayAr.append([day,count_day,start_idx,curr_idx])
    return count_dayAr



In [133]:
def find_heading(temp_article):
    article = temp_article[0].split('\n')[1]
    article += '\n'
    for article_full in temp_article[1:len(temp_article)-1]:
        article += article_full.split('\n')[2]
        article += '\n'
    return article

In [222]:
dist = 'allahabad'

In [248]:
#dont add zeros infront of months 
year = 2012
month = 2

In [249]:
article_list = articleList()

In [250]:
article_heading = find_heading(article_list).split('\n')

In [251]:
article_heading

['Will set up schools, hospitals in Maya parks: Mulayam',
 'Accused seeks bail in fake Allahabad Law University degree racket case',
 'weather',
 'HC upset over snail pace probe in NRHM scam',
 "Procedural gaps halt Uma Bharti's rath yatra",
 'UP polls: Congress puts reforms over freebies',
 'No takers for Diggy Raja',
 'Minimum temperature may dip further',
 'We will name schools after Ambedkar, says Mulayam Singh Yadav',
 '249 candidates in fray after scrutiny',
 'No takers for Digvijay Singh',
 'Will set up hospitals, schools in Maya parks, promises Mulayam',
 'Lyngdoh for greater role to citizens, civic bodies',
 "HC dismisses Shanti Bhushan's plea",
 "HC rejects PIL for covering of Nehru, Indira, Rajiv's statues",
 'Ambedkar schools,hospitals to replace Maya monuments:\xc2\xa0SP',
 '\xc2\x91I was like a pawn guarding wazir\xc2\x92: Kushwaha points fingers at\xc2\xa0Maya',
 'HC rejects PIL for covering of Nehru,Indira,Rajiv\xe2\x80\x99s statues',
 "'Need to grant IIT status to IT-B

In [252]:
article = ''
for heading in article_heading:
    article += heading 

In [253]:
rake_object.run(article)

[('polls', 2.227272727272727)]

In [254]:
len(article_list)

411

In [255]:
[keywords,keyword_0,article] = extractKeywords2(article_list,0,len(article_list))

now sending the article to rake to extract keywords.... 
now making the keywords arranged for you


In [256]:
df_keywords = pd.DataFrame(keywords)

In [257]:
df_keywords.columns = ['keyword','score']

In [258]:
df_keywords

,keyword,score
0,il&fs transportation networks,14.222222
1,khatu shyam prabhu,8.625000
2,asahi songwon colors,8.600000
3,quami ekta dal,8.576271
4,havy rain/snowfall warning,8.500000
5,apl apollo tubes,8.250000
6,twilight litaka pharma,8.116279
7,gremach infrastructure equipments,8.072727
8,deriving political mileage,8.010938
9,mln medical college,7.982979


In [259]:
df_keywords.to_csv('keyword_temp.csv')

In [41]:
df_keywords.to_csv('keywords_thiruvallur_nov2015.csv')

In [ ]:
keyword_0

In [ ]:
len(article_list)

In [ ]:
countDays(article_list)